# Prototipo de Dashboard (Fase 2 - Versión Final)

Este notebook simula las pruebas realizadas para la creación de `main_dashboard.py`. 

**Objetivo:** Prototipar la lógica completa del dashboard, incluyendo:
1.  Carga de datos de `procesados_movies.csv` (que incluye `tmdbId`).
2.  Llamadas a la API de TMDB para obtener URLs de pósteres.
3.  Lógica de filtrado y ordenamiento (Puntaje vs. Popularidad).
4.  Lógica de truncamiento de títulos para el layout.
5.  Prueba de todas las visualizaciones de Plotly.

## 1. Importaciones y Configuración Global

In [ ]:
import pandas as pd
import plotly.express as px
import os
import requests
import math
from IPython.display import display, HTML # Para mostrar imágenes en Jupyter

# Define la clave de la API de TMDB (v3 auth)
TMDB_API_KEY = "c8f4aca1c7dedc6184e0cf3f98e2665e"

# Define las rutas (sube un nivel '..' desde 'script_pelicula')
BASE_DIR = os.path.abspath('..') 
DATA_PROCESS_DIR = os.path.join(BASE_DIR, 'data', 'process')
PROCESSED_FILE = os.path.join(DATA_PROCESS_DIR, 'procesados_movies.csv')

# Define la URL del póster por defecto (con la proporción 2:3 correcta)
DEFAULT_POSTER = "https://via.placeholder.com/500x750.png?text=Poster+No+Disponible"

## 2. Funciones de Carga y Helpers

Define las mismas funciones que usa el script `main_dashboard.py`.

In [ ]:
def load_data(file_path):
    """
    Esta función carga el archivo CSV procesado de la Fase 1.
    """
    try:
        df = pd.read_csv(file_path, encoding='latin1')
        df['tmdbId'] = pd.to_numeric(df['tmdbId'], errors='coerce')
        return df
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo 'procesados_movies.csv' en {file_path}")
        return None

In [ ]:
def get_dynamic_columns(df):
    """
    Esta función extrae dinámicamente las columnas de género (One-Hot) 
    y las de año (Pivot) del DataFrame.
    """
    base_cols = ['movieid', 'title', 'genres', 'rating_promedio', 'rating_conteo', 'tag', 'tmdbId']
    year_cols = [col for col in df.columns if col.isdigit() and len(col) == 4]
    genre_cols = [col for col in df.columns if col not in base_cols and col not in year_cols]
    return genre_cols, year_cols

In [ ]:
def get_poster_url(tmdb_id):
    """
    Esta función llama a la API de TMDB para obtener la URL del póster.
    """
    if pd.isna(tmdb_id):
        return DEFAULT_POSTER
        
    url = f"https://api.themoviedb.org/3/movie/{int(tmdb_id)}?api_key={TMDB_API_KEY}"
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
        if data.get("poster_path"):
            return f"https://image.tmdb.org/t/p/w500{data['poster_path']}"
        else:
            return DEFAULT_POSTER
    except requests.exceptions.RequestException:
        return DEFAULT_POSTER

## 3. Carga de Datos

Ejecuta las funciones de carga y muestra un `head()` para verificar.

In [ ]:
df_procesado = load_data(PROCESSED_FILE)

if df_procesado is not None:
    genre_columns, year_columns = get_dynamic_columns(df_procesado)
    print(f"Datos cargados: {df_procesado.shape}")
    print(f"{len(genre_columns)} géneros encontrados.")
    print(f"{len(year_columns)} años encontrados.")
    display(df_procesado.head())

## 4. Simulación de Filtros (Widgets)

En Streamlit, estos valores vienen de los widgets. Aquí se simula la interacción del usuario definiendo las variables a mano.

In [ ]:
# Simulación de los widgets de la barra lateral

# Simulación del st.multiselect de Géneros
selected_genres = ['Action', 'SciFi']

# Simulación del st.slider de Rating
rating_slider = (3.5, 5.0) # (min, max)

# Simulación del st.slider de Conteo de Ratings
total_ratings_slider = 1000 # Mínimo 1000 calificaciones

# Simulación de la nueva opción de ordenamiento
sort_by = "Puntaje (Mejor Calificadas)" # Opciones: "Puntaje (Mejor Calificadas)" o "Popularidad (Más Votadas)"

print(f"Simulando filtros:")
print(f"- Géneros: {selected_genres}")
print(f"- Rango de Rating: {rating_slider}")
print(f"- Mínimo de Calificaciones: {total_ratings_slider}")
print(f"- Ordenar por: {sort_by}")

## 5. Prueba de la Lógica de Filtrado y Ordenamiento

In [ ]:
df_filtrado = df_procesado.copy()

# Aplica los filtros simulados al DataFrame
df_filtrado = df_filtrado[
    (df_filtrado['rating_promedio'] >= rating_slider[0]) &
    (df_filtrado['rating_promedio'] <= rating_slider[1])
]
df_filtrado = df_filtrado[
    df_filtrado['rating_conteo'] >= total_ratings_slider
]
if selected_genres:
    for genre in selected_genres:
        df_filtrado = df_filtrado[df_filtrado[genre] == 1]

# Aplica la lógica de ordenamiento según el 'sort_by' simulado
if sort_by == "Popularidad (Más Votadas)":
    df_filtrado = df_filtrado.sort_values(by='rating_conteo', ascending=False)
    sort_label = "Popularidad"
else:
    df_filtrado = df_filtrado.sort_values(by='rating_promedio', ascending=False)
    sort_label = "Puntaje"

print(f"Filas antes: {len(df_procesado)} | Filas después: {len(df_filtrado)}")
print(f"Ordenado por: {sort_label}")
display(df_filtrado.head())

## 6. Simulación Pestaña 'Análisis de Datos'

Prueba de todos los gráficos para la pestaña 'Análisis de Datos'.

### 6.1 Gráfico de Barras: Top 10 Popularidad

In [ ]:
# Este gráfico siempre se ordena por 'rating_conteo'.
df_top10_pop = df_filtrado.nlargest(10, 'rating_conteo')
fig_bar = px.bar(
    df_top10_pop, x='rating_conteo', y='title', orientation='h',
    title='Top 10 Películas más Populares', hover_data=['rating_promedio', 'genres']
)
fig_bar.update_layout(yaxis={'categoryorder':'total ascending'})
# En Jupyter, se usa .show() para mostrar el gráfico.
fig_bar.show()

### 6.2 Gráfico de Dispersión: Rating vs. Popularidad

In [ ]:
fig_scatter = px.scatter(
    df_filtrado, x='rating_conteo', y='rating_promedio',
    title='Rating vs. Popularidad', hover_data=['title', 'genres'],
    color='rating_promedio', size='rating_conteo' 
)
fig_scatter.show()

### 6.3 Gráfico de Líneas: Evolución del Rating

Simula que el usuario selecciona una película de la lista.

In [ ]:
if not df_filtrado.empty:
    # Simula la selección de la primera película de la lista filtrada.
    selected_movie_title = df_filtrado.iloc[0]['title']
    print(f"Simulando selección de: {selected_movie_title}")

    movie_data = df_filtrado[df_filtrado['title'] == selected_movie_title].iloc[0]
    evolution_data = movie_data[year_columns][movie_data[year_columns] > 0]
    
    if not evolution_data.empty:
        df_evo = pd.DataFrame({
            'Anio': evolution_data.index.astype(int),
            'Rating Promedio': evolution_data.values
        })
        fig_line = px.line(
            df_evo, x='Anio', y='Rating Promedio',
            title=f"Evolución del Rating para: {selected_movie_title}", markers=True
        )
        fig_line.show()
    else:
        print(f"No hay datos de evolución para {selected_movie_title}")
else:
    print("DataFrame filtrado vacío.")

## 7. Simulación Pestaña 'Explorador de Pósteres'

Prueba la lógica de truncamiento de título y la llamada a la API para el Top 20.

In [ ]:
if not df_filtrado.empty:
    # Toma el Top 20 del df_filtrado (ya ordenado).
    df_paginado = df_filtrado.head(20)
    
    # Prepara una lista de HTML para mostrar los pósteres.
    html_output = ""

    for i, row in enumerate(df_paginado.itertuples()):
        poster_url = get_poster_url(row.tmdbId)
        
        # Lógica de truncamiento de título para alinear la cuadrícula.
        title = row.title
        if len(title) > 30:
            title = title[:30] + "..."
        
        # Crea una "tarjeta" HTML para simular la vista de Streamlit.
        html_output += (f"<div style='display:inline-block; width: 200px; margin: 10px; vertical-align: top; text-align: center; border: 1px solid #333;'>"
                        f"  <img src='{poster_url}' style='width: 100%; height: 300px; object-fit: cover;'>"
                        f"  <h4 style='font-size: 14px; height: 40px;'>{title}</h4>"
                        f"  <p style='font-size: 12px;'>Rating: {row.rating_promedio:.1f} ⭐ ({row.rating_conteo:,} votos)</p>"
                        f"</div>")

    # Usa display(HTML(...)) para renderizar el HTML en Jupyter.
    display(HTML(html_output))
else:
    print("DataFrame filtrado vacío.")